In [1]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
#from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications import mobilenet
from keras.layers import Dense, Input, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation
from keras.models import Sequential
from keras.layers import Dense, Input, Activation
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

/home/mdmhafi/.conda/envs/tf_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def createModel():
    input_shape = (400,400,3)
    model = Sequential()
    # Block 1
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
 
    # Block 2
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
 
    # Block 3
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
 
    # Block 4
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
 
    # Block 5
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    # Block 6
    #model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    #model.add(Conv2D(32, (3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    # FC1
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.25))
    
    # FC2
    model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.25))
    
    #FC3
    model.add(Dense(128, activation='relu'))
    #model.add(Dropout(0.25))
    
    #output layer
    model.add(Dense(2, activation='softmax'))
     
    return model

In [3]:
model = createModel()
#model.summary()

from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [4]:
import glob
train_imgs_r = glob.glob('data/train/red/*')
train_imgs_nr = glob.glob('data/train/notred/*')
val_imgs_r = glob.glob('data/val/red/*')
val_imgs_nr = glob.glob('data/val/notred/*')
print ('Train red dataset size : ' , len(train_imgs_r))
print ('Train notred dataset size : ' , len(train_imgs_nr))
print ('Val red dataset size : ' , len(val_imgs_r))
print ('Val notred dataset size : ' , len(val_imgs_nr))

total_train = len(train_imgs_r) + len(train_imgs_nr)
total_val   = len(val_imgs_r) + len(val_imgs_nr)


print ('Total training dataset' , total_train)
print ('Total validation dataset' , total_val)


img_width, img_height = 400, 400
path='data/'
train_data_dir = path + "train"
validation_data_dir =  path + "val"

batch_size = 8
epochs = 100

img_rows, img_cols = img_height, img_width
# number of convolutional filters to use
#nb_filters = 32
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

num_classes = 2

print ('OK')

Train red dataset size :  631
Train notred dataset size :  161
Val red dataset size :  101
Val notred dataset size :  31
Total training dataset 792
Total validation dataset 132
OK


In [5]:
# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = "nearest",
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.0,
                                  height_shift_range=0.0,
                                  rotation_range=30)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size,
                                                    class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size = (img_height, img_width),
                                                        class_mode = "categorical")
print ('OK')

Found 792 images belonging to 2 classes.
Found 132 images belonging to 2 classes.
OK


In [6]:
# Save the model according to the conditions
checkpoint = ModelCheckpoint("model_vgg_2.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)

early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=10, 
                      verbose=1, 
                      mode='auto')

print ('OK')

OK


In [7]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Train the model
history = model.fit_generator(train_generator,
                    steps_per_epoch  = total_train,
                    epochs = epochs,
                    validation_data = validation_generator,
                    validation_steps = total_val,
                    callbacks = [checkpoint, early])

Epoch 1/100
792/792 [==============================] - 331s 418ms/step - loss: 0.3256 - acc: 0.8668 - val_loss: 0.1975 - val_acc: 0.9130

Epoch 00001: val_acc improved from -inf to 0.91304, saving model to model_vgg_2.h5
Epoch 2/100
792/792 [==============================] - 325s 410ms/step - loss: 0.1775 - acc: 0.9141 - val_loss: 0.1931 - val_acc: 0.9108

Epoch 00002: val_acc did not improve from 0.91304
Epoch 3/100
792/792 [==============================] - 324s 409ms/step - loss: 0.2009 - acc: 0.9021 - val_loss: 0.1898 - val_acc: 0.9145

Epoch 00003: val_acc improved from 0.91304 to 0.91447, saving model to model_vgg_2.h5
Epoch 4/100
792/792 [==============================] - 326s 411ms/step - loss: 0.1785 - acc: 0.9127 - val_loss: 0.1852 - val_acc: 0.9153

Epoch 00004: val_acc improved from 0.91447 to 0.91533, saving model to model_vgg_2.h5
Epoch 5/100
792/792 [==============================] - 326s 412ms/step - loss: 0.1800 - acc: 0.9115 - val_loss: 0.1905 - val_acc: 0.9108

Epoch

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
def process_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(400, 400))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

import glob
not_reds = glob.glob('data/test/notred/*')
for img in not_reds:
    img_tensor = process_image(img)
    pred = model.predict(img_tensor)
    predicted_class = np.argmax(pred, axis=1)
    
    print ('Pedicted classes: ', predicted_class[0] )